Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ERROR:pyngrok.process.ngrok:t=2025-07-26T08:41:25+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-26T08:41:25+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [3]:
!pip install -q transformers peft flask pyngrok

# Step 2: Import everything
from flask import Flask, request, jsonify
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Step 3: Set paths to your model files
from google.colab import drive
drive.mount('/content/drive')

base_model_path = "/content/drive/MyDrive/TinyLlama-1.1B-Chat-v1.0"  # Upload base model to /content/TinyLlama
adapter_path = "/content/drive/MyDrive/tinyllama-finetuned1"       # Upload fine-tuned adapter to /content/Adapter

# Step 4: Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
base_model = AutoModelForCausalLM.from_pretrained(base_model_path)
model = PeftModel.from_pretrained(base_model, adapter_path)
model = model.merge_and_unload()






Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Step 5: Set up Flask app
app = Flask(__name__)
# run_with_ngrok(app)  # Expose API via ngrok
@app.route("/generate", methods=["POST"])
def generate_response():
    data = request.get_json()
    prompt = data.get("prompt", "")

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.2,
    eos_token_id=2
    # eos_token_id=tokenizer.eos_token_id  # if available
)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return jsonify({"response": response})

In [14]:
# Step 6: Run the app
!ngrok config add-authtoken 2xYIg3Za6l9LeGlS2DGt2EbA88N_5qu3HXPd66NCAHv95cNAp

public_url = ngrok.connect(5000)
print("🌐 Public URL:", public_url)

app.run(port=5000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌐 Public URL: NgrokTunnel: "https://86d1f8c13f08.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Jul/2025 09:41:27] "POST /generate HTTP/1.1" 200 -
